In [26]:
# Enterprise Search Cookbook

## Enterprise Search is a foundation for building Retrieval-Augmented Generation (RAG) pipelines. It provides accurate answers based on your documents and offers a simple API for indexing and querying document collections.

In [3]:
## 2. Setup and Prerequisites

# First, let's set up our environment:
# Ensure you have conda installed in your system before running this cell.

In [4]:
## 3. Configuration

# Let's set up the configuration:

# !cp .env.example .env

# Edit .env file with your settings
# You can use !echo "KEY=VALUE" >> .env to add settings

# Start Redis and Qdrant services
!docker-compose -f docker/docker-compose.yml down
!docker-compose -f docker/docker-compose.yml up -d redis qdrant

# Setup LLM (Ollama example)
!docker-compose -f docker/docker-compose-ollama.yml up -d

Stopping qdrant_vector_store ... 
Stopping redis_doc_store     ... 
Removing qdrant_vector_store ... 
Removing redis_doc_store     ... 
Removing network docker_default32mdone
Creating network "docker_default" with the default driver
Creating qdrant_vector_store ... 
Creating redis_doc_store     ... 
ting qdrant_vector_store ... doneWARNING: Found orphan containers (qdrant_vector_store, redis_doc_store) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up.
ollama is up-to-date


In [5]:
## 4. Configuring and Running the Pipeline

# First, let's update some important configuration settings
from llamasearch.settings import config
from llamasearch.pipeline import Pipeline, setup_global_embed_model

# Override important settings
config.application.data_path = "./data/test_docs/"  # Path to your documents
config.vector_store_config.collection_name = "my_test_collection"
# config.embedding.model = "sentence-transformers/all-MiniLM-L6-v2"  # A smaller, faster model for testing
# config.vector_store_config.vector_size = 384 # Update vector dims to match the embed model dims

# Set up the global embedding model
global_embed_model = setup_global_embed_model(config)

# Now, let's run the pipeline:
import asyncio
import nest_asyncio

async def run_pipeline():
    tenant_id = "test_tenant"
    pipeline = Pipeline(config, tenant_id, global_embed_model)
    await pipeline.setup()
    return pipeline

# Use this for Jupyter notebooks
nest_asyncio.apply()

pipeline = asyncio.get_event_loop().run_until_complete(run_pipeline())

print("Pipeline setup complete. Ready for queries!")


/home/deval/Document/Work/miniconda3/envs/llamasearch_merge_test/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in Eval has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(



Path Configuration:
BASE_PATH:     /home/deval/Documents/Work/Deval/ES/LlamaSearch_final/LlamaSearch
--------------------------------------------------------------------------------
Resolved Paths:
Config File:   /home/deval/Documents/Work/Deval/ES/LlamaSearch_final/LlamaSearch/config/config.dev.yaml
Data Path:     /home/deval/Documents/Work/Deval/ES/LlamaSearch_final/LlamaSearch/data/test_docs
Log Directory: /home/deval/Documents/Work/Deval/ES/LlamaSearch_final/LlamaSearch/data/app/logs



2024-10-15 15:37:33,180 - [INFO] - Using embedding model: Alibaba-NLP/gte-Qwen2-1.5B-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-10-15 15:37:40,743 - [INFO] - Running model ajindal/llama3.1-storm:8b on http://localhost:11434
2024-10-15 15:37:40,744 - [INFO] - Multi tenancy: True
2024-10-15 15:37:40,744 - [INFO] - Setting up Qdrant index...
2024-10-15 15:37:40,773 - [DEBUG] - Collection names: []
2024-10-15 15:37:40,774 - [INFO] - Collection my_test_collection does not exist. Creating new collection...
Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


Creating vector store for collection my_test_collection


2024-10-15 15:37:42,366 - [INFO] - Qdrant index setup completed.
2024-10-15 15:37:42,368 - [INFO] - Setting up Docstore...
2024-10-15 15:37:42,370 - [INFO] - Docstore setup completed.
2024-10-15 15:37:42,372 - [INFO] - Setting up Parser...
2024-10-15 15:37:42,571 - [INFO] - Parser setup completed.
2024-10-15 15:37:42,572 - [INFO] - Setting up Documents...
2024-10-15 15:37:43,586 - [INFO] - Documents setup completed.
2024-10-15 15:37:43,586 - [INFO] - Setting up Index creation...
2024-10-15 15:37:43,587 - [INFO] - Index creation setup completed.
2024-10-15 15:37:43,587 - [INFO] - Setting up Ingestion pipeline...
2024-10-15 15:37:43,615 - [INFO] - Ingesting 43 nodes for 38 chunks
2024-10-15 15:37:43,615 - [INFO] - Adding nodes to index...
Generating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.20s/it]
2024-10-15 15:37:50,062 - [INFO] - Inges

Pipeline setup complete. Ready for queries!


In [3]:
## 5. Querying and Results

# Let's perform a query:

async def perform_query(pipeline, query):
    response = await pipeline.perform_query_async(query)
    return response

query = "How does attention mechanism work in transformer architecture?"
response = asyncio.run(perform_query(pipeline, query))

print("Query:", query)
print("Response:", response)

Query: How does attention mechanism work in transformer architecture?
Response: The attention mechanism works by computing a weighted sum of values based on a compatibility function between a query and a set of key-value pairs. In the Transformer architecture, this is done using scaled dot-product attention, where the input consists of queries and keys of dimension dk, and values of dimension dv. The attention function is computed as softmax(QKT√dk)V, where Q, K, and V are matrices of queries, keys, and values, respectively.


In [4]:
# Pretty print the context
pipeline.pretty_print_context(response)


--- Document Information ---
+-------------------------------+-----------------+--------------------------------------+
| File Name                     | Last Modified   | Doc ID                               |
+===============================+=================+======================================+
| attention_is_all_you_need.pdf | 2024-10-04      | aa1f72b2-3b1d-444f-b367-adb17b8fb29b |
+-------------------------------+-----------------+--------------------------------------+


In [5]:
# Cleanup
asyncio.run(pipeline.cleanup())

print("Pipeline cleaned up.")

Pipeline cleaned up.
